In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import random
import cchardet
import scrapy

### Scrape data from `Cars.com`

In [17]:
website='https://www.cars.com/shopping/results/?page=1&page_size=20&dealer_id=&keyword=&list_price_max=&list_price_min=&makes[]=mercedes_benz&maximum_distance=all&mileage_max=&sort=best_match_desc&stock_type=cpo&year_max=&year_min=&zip='

In [18]:
request=requests.get(website)


In [19]:
soup=BeautifulSoup(request.text,'lxml')

In [20]:
## to select all the 20 entries in a page
page_result=soup.find_all('div',{'class':'vehicle-card'})


In [21]:
# to get order details
web2=page_result[1].find('a',{'class':'vehicle-card-visited-tracking-link'})['href']
car_details='https://www.cars.com' + web2
request2=requests.get(car_details)
soup2=BeautifulSoup(request2.text,'html5lib')


In [223]:
# How to extract consumer reviews
soup2.find('div',{'class':'sds-rating sds-rating--big'}).find('span',{'class':"sds-rating__count"}).text

'4.6'

In [59]:
# How to get reviews
soup2.find('div',{'class':'sds-rating sds-rating--big'}).find('a',{'class':'sds-rating__link sds-button-link'}).text

'(72 reviews)'

In [41]:
x=soup2.find('section',{'class':"sds-page-section basics-section"}).find('dl',{'class':'fancy-description-list'})


In [48]:
# to get car color
x.find_all('dd')[1]

<dd> Black </dd>

### Main code

In [9]:
## car name
def extract_car_page(page,car):
    website='https://www.cars.com/shopping/results/?page='+str(page)+'&page_size=30&dealer_id=&keyword='+\
                 '&list_price_max=&list_price_min=&makes[]='+car+'&maximum_distance=all&mileage_max=&'+\
                 'sort=best_match_desc&stock_type=cpo&year_max=&year_min=&zip='
    request=requests.get(website)
    soup=BeautifulSoup(request.text,'lxml')
    page_result=soup.find_all('div',{'class':'vehicle-card'})
    
    return page_result

def more_info(car_info):
    web2=car_info.find('a',{'class':'vehicle-card-visited-tracking-link'})['href']
    car_details='https://www.cars.com' + web2
    request2=requests.get(car_details)
    soup2=BeautifulSoup(request2.text,'lxml')
    
    return soup2
 

def parser(cars=['mercedes_benz','toyota','ford'],page_num=5):
    
    entry=[] # a container for dicts storing extracted items
       
    for car in cars:
        for page in random.choices(range(10),k=page_num):
        
            page_result=extract_car_page(page,car)
    
            for result in page_result:
                try:
                    car_name=result.find('h2',attrs={'class':'title'}).text
                except:
                    car_name=np.NaN
    
                try:
                    mileage=result.find('div',attrs={'class':'mileage'}).text
                except:
                    mileage=np.NaN
        
                try:
                    price=result.find('span',attrs={'class':'primary-price'}).text
                except:
                    price=np.NaN
    
                try:
                    dealer_name=result.find('div',attrs={'class':'dealer-name'}).text.strip()
                except:
                    dealer_name=np.NaN
    
                try:
                    ratings=result.find('span',attrs={'class':'sds-rating__count'}).text
                except:
                    ratings=np.NaN
    
                try:
                    reviews=result.find('span',attrs={'class':'sds-rating__link'}).text
                except:
                    reviews=np.NaN
            
        
                # getting more information on specfic car         
                soup2=more_info(result)

                try:
                    consumer_review=soup2.find('div',{'class':'sds-rating sds-rating--big'}).\
                                       find('span',{'class':"sds-rating__count"}).text
                    review_num=soup2.find('div',{'class':'sds-rating sds-rating--big'}).\
                                    find('a',{'class':'sds-rating__link sds-button-link'}).text
                except:
                    consumer_review=np.NaN
                    review_num=np.NaN
            
    
                basic_data=soup2.find('section',{'class':"sds-page-section basics-section"}).\
                              find('dl',{'class':'fancy-description-list'})
                try:
                    Ext_color=basic_data.find_all('dd')[0].text
                    int_color=basic_data.find_all('dd')[1].text
                    drive_train=basic_data.find_all('dd')[2].text
                    fuel_type=basic_data.find_all('dd')[4].text
                    transmission=basic_data.find_all('dd')[5].text
                    engine=basic_data.find_all('dd')[6].text
                except:
                    Ext_color=np.NaN
                    int_color=np.NaN
                    drive_train=np.NaN
                    fuel_type=np.NaN
                    transmission=np.NaN
                    engine=np.NaN
                
                item={'Car_name':car_name,'mileage':mileage,'dealer_name':dealer_name,\
                      'Service_ratings':ratings,'Service_reviews':reviews,'consumer_ratings':consumer_review,\
                      'consumer_reviews':review_num,'price':price,'Ext_color':Ext_color,\
                      'Int_color':int_color,'drive_train':drive_train,'fuel_type':fuel_type,\
                      'transmission':transmission,'engine':engine
                     }
            
                entry.append(item)
            
    return entry
    

    
            
        
                
          
            
        
    

In [10]:
data=pd.DataFrame(parser(page_num=1))

In [12]:
#data cleaning
data=data.dropna().reset_index(drop=True)
data['mileage']=data['mileage'].str.strip(' mi.').str.replace(',','').astype(int)
data['price']=data['price'].str.strip('$').str.replace(',','').astype(int)
data['Service_reviews']=data['Service_reviews'].str.strip('(').str.strip(' reviews)').str.replace(',','').astype(int)
data['consumer_reviews']=data['consumer_reviews'].str.strip('(').str.strip(' reviews)').str.replace(',','').astype(int)
data['Service_ratings']=data['Service_ratings'].astype(float)

In [11]:
#data.to_csv('first_car_data.csv')

In [13]:
data['engine'].value_counts()

2.0L I4 16V GDI DOHC Turbo                  26
2.5L I4 16V PDI DOHC                        12
2.7L V6 24V PDI DOHC Twin Turbo              9
1.8L I4 16V MPFI DOHC                        6
3.0L V6 24V GDI DOHC Twin Turbo              4
1.5L I4 16V GDI DOHC Turbo                   4
3.5L V6 24V MPFI DOHC                        3
3.5L V6 24V PDI DOHC                         3
3.5L V6 24V PDI DOHC Twin Turbo              3
3.5L V6 24V GDI DOHC Twin Turbo              2
2.3L I4 16V GDI DOHC Turbo                   1
5.0L V8 32V PDI DOHC                         1
Regular Unleaded I-4 2.5 L/152               1
2.7L V6 24V GDI DOHC Twin Turbo              1
1.8L I4 16V MPFI DOHC Hybrid                 1
Regular Unleaded V-6 3.5 L/211               1
2.7L I4 16V MPFI DOHC                        1
2.0L I4 16V MPFI DOHC                        1
2.5L I4 16V PDI DOHC Hybrid                  1
4.0L V6 24V MPFI DOHC                        1
4.6L V8 32V MPFI DOHC                        1
2.5L I4 16V M